## Import

In [23]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import os
import numpy as np
from statannotations.Annotator import Annotator
from lifelines import KaplanMeierFitter
from lifelines.utils import median_survival_times

In [24]:
df = pd.read_excel("./data/df_for_analysis.xlsx",index_col=0)

## Utils function

In [25]:
def keep_weight_post_infection(x,weight_end = "weight_T13"):
    """
    Given a DataFrame `x`, returns a Series containing the weights post-infection.

    *Arguments*
    - x: DataFrame containing the data.
    - weight_end: Column name of the last weight measurement.

    *Returns*
    - shifted_series: Series containing the weights post-infection.
    """
    # Extract relevant columns from the DataFrame
    dates = x['Dates']
    t_infection = x['Time_infection']
    datas = x['weight_T_infection':weight_end]

    # Find the date closest to the infection time
    new_time_infection = dates[dates <= t_infection][-1]
    location_of_TI = dates.get_loc(new_time_infection)

    # Return the original series if the data at the infection time is NaN
    if np.isnan(datas[location_of_TI]):
        return datas
    
    # Shift the values of the input series by the specified index
    shifted_series = pd.Series([np.nan] * len(datas), index=datas.index)
    if location_of_TI == 0:
        return datas
    else:
        shifted_series[:-location_of_TI] = datas.values.tolist()[location_of_TI:]
    # Shift the values of the input series by the specified index
    return shifted_series

## Transform data to long format

Keep only data that are after the time of infeciton

In [26]:
# change dates column to datetimindex and transform weight datas to numeric only
df.loc[:,"weight_T_infection":"weight_T13"] = df.loc[:,"weight_T_infection":"weight_T13"].apply(pd.to_numeric,errors='coerce')
serie_dates = df['Time_point'].apply(lambda x: pd.to_datetime(x.split(','),dayfirst=True))
df['Dates'] = serie_dates
data = df.apply(lambda x: keep_weight_post_infection(x),axis=1)
data

C:\Users\ma7054\AppData\Local\Temp\ipykernel_19732\2938565056.py:2: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df.loc[:,"weight_T_infection":"weight_T13"] = df.loc[:,"weight_T_infection":"weight_T13"].apply(pd.to_numeric,errors='coerce')


,weight_T_infection,weight_T1,weight_T2,weight_T3,weight_T4,weight_T5,weight_T6,weight_T7,weight_T8,weight_T9,weight_T10,weight_T11,weight_T12,weight_T13
0,23.92,21.72,20.96,19.38,18.16,16.44,15.49,15.44,15.05,NaN,NaN,NaN,NaN,NaN
1,21.40,19.45,18.84,17.82,16.80,15.02,14.14,14.40,14.73,NaN,NaN,NaN,NaN,NaN
2,22.56,21.45,20.83,18.67,16.82,15.30,14.74,14.80,14.42,NaN,NaN,NaN,NaN,NaN
3,20.39,18.69,16.60,15.58,14.17,13.54,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,23.72,21.74,20.29,19.56,18.50,16.80,16.78,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2352,27.70,25.00,22.80,21.10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2353,26.10,24.90,22.60,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2354,24.60,25.80,25.70,24.90,22.80,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2355,27.40,27.30,28.20,27.80,27.70,27.80,27.60,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Normalization by weight at T infection and replace the normalize data into the original dataframe

In [27]:

normalize = data.div(data['weight_T_infection'],axis=0)*100
df_normalize = df.copy()
df_normalize.loc[:,"weight_T_infection":"weight_T13"] = normalize

df_normalize['min_weight'] = df_normalize.loc[:,"weight_T_infection":"weight_T13"].min(axis=1)
df_normalize['t_origin'] = df_normalize['time_original']
df_normalize

,Mouse_ID,ID_Experiment,Cage,Strain,Date,Experiment,Group,Group_info,H0,Pre_traitment,...,survival_0.06,time_0.05,survival_0.05,time_original,survival_original,max_loss_weight_percentage,exp,sub_exp,min_weight,t_origin
0,TRO-05432,ID_001,A,BALB/cByJ,2014-06-05,Candida/Propionate,1A,Propionate / 2*10^5,1,propionate,...,1,1.5,1,9.0,1,0.629181,1,A,62.918060,9.0
1,TRO-05433,ID_001,A,BALB/cByJ,2014-06-05,Candida/Propionate,1A,Propionate / 2*10^5,1,propionate,...,1,1.5,1,9.0,1,0.660748,1,A,66.074766,9.0
2,TRO-05434,ID_001,A,BALB/cByJ,2014-06-05,Candida/Propionate,1A,Propionate / 2*10^5,1,propionate,...,1,2.5,1,9.0,1,0.639184,1,A,63.918440,9.0
3,TRO-05435,ID_001,A,BALB/cByJ,2014-06-05,Candida/Propionate,1A,Propionate / 2*10^5,1,propionate,...,1,1.5,1,6.0,1,0.664051,1,A,66.405101,6.0
4,TRO-05456,ID_001,B,BALB/cByJ,2014-06-05,Candida/Propionate,1A,Propionate / 2*10^5,1,propionate,...,1,1.5,1,7.0,1,0.707420,1,A,70.741990,7.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2352,TRO-028337,ID_096,ETRO-01911,C57BL/6J,2023-03-03,Pneumococcus/Training/Cross-fostering/male,3,D. Zy-Zy,1,training/cross-fostering,...,1,1.5,1,5.0,1,0.761733,3,no,76.173285,5.0
2353,TRO-028338,ID_096,ETRO-01911,C57BL/6J,2023-03-03,Pneumococcus/Training/Cross-fostering/male,3,D. Zy-Zy,1,training/cross-fostering,...,1,2.5,1,4.0,1,0.865900,3,no,86.590038,4.0
2354,TRO-028339,ID_096,ETRO-01911,C57BL/6J,2023-03-03,Pneumococcus/Training/Cross-fostering/male,3,D. Zy-Zy,1,training/cross-fostering,...,1,5.5,1,6.0,1,0.926829,3,no,92.682927,6.0
2355,TRO-028342,ID_096,ETRO-01911,C57BL/6J,2023-03-03,Pneumococcus/Training/Cross-fostering/male,3,D. Zy-Zy,1,training/cross-fostering,...,0,8.0,0,8.0,0,0.996350,3,no,99.635036,8.0


Find only releavent columns


In [28]:
columns = df_normalize.loc[:,"weight_T_infection":"weight_T13"].columns.tolist()
columns_index = df_normalize.loc[:, ~df_normalize.columns.isin(columns)]
column_time = [n for n in df_normalize.columns.tolist() if "time_" in n]
column_time = [column_time[-1]] + column_time[:-1]
columns_index_time = ['ID_Experiment','Mouse_ID','Date','Infection','Group','exp','survival_original','t_origin']

Transform to tidy data for releavent columns

In [29]:
df_longer_weight = df_normalize.melt(id_vars=columns_index_time,value_vars=columns,var_name="Time",value_name="weight")
df_longer_weight['Time'] = df_longer_weight['Time'].apply(lambda x: "".join(x.split("_")[1:]))
df_longer_weight

,ID_Experiment,Mouse_ID,Date,Infection,Group,exp,survival_original,t_origin,Time,weight
0,ID_001,TRO-05432,2014-06-05,C. albicans,1A,1,1,9.0,Tinfection,100.0
1,ID_001,TRO-05433,2014-06-05,C. albicans,1A,1,1,9.0,Tinfection,100.0
2,ID_001,TRO-05434,2014-06-05,C. albicans,1A,1,1,9.0,Tinfection,100.0
3,ID_001,TRO-05435,2014-06-05,C. albicans,1A,1,1,6.0,Tinfection,100.0
4,ID_001,TRO-05456,2014-06-05,C. albicans,1A,1,1,7.0,Tinfection,100.0
...,...,...,...,...,...,...,...,...,...,...
32993,ID_096,TRO-028337,2023-03-03,S. pneumoniae,3,3,1,5.0,T13,NaN
32994,ID_096,TRO-028338,2023-03-03,S. pneumoniae,3,3,1,4.0,T13,NaN
32995,ID_096,TRO-028339,2023-03-03,S. pneumoniae,3,3,1,6.0,T13,NaN
32996,ID_096,TRO-028342,2023-03-03,S. pneumoniae,3,3,0,8.0,T13,NaN


save the datas

In [30]:
df_longer_weight.to_excel("./data/df_long_format_for_analysis.xlsx")

## Weight loss by day of infection

In [31]:
df_longer_weight.groupby(["Infection",'Time'])['weight'].agg(['mean',"median","count","std"])

mean      median  count        std
Infection     Time                                                
C. albicans   T1           93.279207   93.953488    251   3.862355
              T10          88.030905   90.481172    117  11.002648
              T11          89.093038   91.262136    109  11.051094
              T12          89.512350   92.839506    103  11.733767
              T13          91.540737   94.016363     36   8.440398
              T2           87.984191   88.369218    234   5.431728
              T3           85.058591   83.976864    232   7.532595
              T4           85.233445   83.467694    230   9.461354
              T5           85.575387   85.300926    224  11.491712
              T6           84.637715   85.512367    153  12.198206
              T7           85.421056   88.261368    138  10.780553
              T8           86.044889   89.134910    134  11.449223
              T9           85.792915   88.042474    124  10.148711
              Tinfection  100.000000  100.000000    252   0.000000
H1N1          T1           98.327784   98.698475    336   4.051930
              T10          90.377142   93.536400    122  13.040054
              T11          89.270627   88.000000     29  16.077912
              T12          83.984279   81.724263     14  13.535254
              T13                NaN         NaN      0        NaN
              T2           94.544187   94.143483    336   5.621029
              T3           91.327386   89.600000    336   7.909140
              T4           88.642446   86.001697    322   9.671188
              T5           86.413411   85.786802    277  11.124952
              T6           85.320512   84.883721    243  12.475680
              T7           88.083057   91.011236    201  12.482347
              T8           90.518798   94.331984    167  12.787637
              T9           90.454694   93.582888    153  12.703328
              Tinfection  100.000000  100.000000    336   0.000000
Listeria      T1           97.493993   97.481092   1038   3.717083
              T10         102.765257  102.544006     16   4.244996
              T11         103.749123  102.834704     16   4.050310
              T12                NaN         NaN      0        NaN
              T13                NaN         NaN      0        NaN
              T2           90.230473   89.417024   1018   5.395146
              T3           87.391896   85.145716    800   8.319945
              T4           89.187612   89.108911    533   9.967525
              T5           91.972894   93.750000    331   9.975515
              T6           94.862222   96.644331    218   9.358876
              T7           95.709996   96.634615    155   7.958578
              T8           96.455673   97.396208     98   8.140584
              T9           95.848173   95.831754     40  10.562874
              Tinfection  100.000000  100.000000   1048   0.000000
S. pneumoniae T1           96.972586   97.461929    700   5.301085
              T10          97.522693  100.469484     81  11.921876
              T11         100.135090  102.204637     68  12.040936
              T12          98.831040   99.130435      7   6.552113
              T13                NaN         NaN      0        NaN
              T2           95.570449   96.875000    697   6.940353
              T3           94.442641   96.995708    635   8.759953
              T4           94.927780   97.714286    482   8.938534
              T5           97.754325  100.000000    374   7.805807
              T6           98.260455  100.453573    302   8.208155
              T7           98.912146  100.588235    263   8.633481
              T8           99.071403  100.993933    200   9.178562
              T9           98.906629  100.978703    152   9.825548
              Tinfection  100.000000  100.000000    721   0.000000

### weight at day 6 non survivor mice

In [32]:
df_dead = df_longer_weight[df_longer_weight['survival_original']==1]
df_survive = df_longer_weight[df_longer_weight['survival_original']==0]
df_dead_T6 = df_dead[df_dead['Time'] == 'T6']

In [33]:
stats = df_dead_T6.groupby(['Infection'])['weight'].agg(['mean','median','count','std'])
stats['ci95_hi'] = stats['median'] + 1.96*stats['std']/np.sqrt(stats['count'])
stats['ci95_low'] = stats['median'] - 1.96*stats['std']/np.sqrt(stats['count'])
stats


,mean,median,count,std,ci95_hi,ci95_low
Infection,,,,,,
C. albicans,72.756838,70.700637,51,7.882190,72.863943,68.537331
H1N1,74.404462,72.855544,66,6.899504,74.520114,71.190974
Listeria,74.637131,74.015748,7,3.225896,76.405525,71.625971
S. pneumoniae,91.887905,91.666667,57,8.133820,93.778274,89.555059


## Median time to death

In [34]:
df = pd.read_excel("./data/df_long_format_for_analysis.xlsx",index_col=0)

In [35]:
df

,ID_Experiment,Mouse_ID,Date,Infection,Group,exp,survival_original,t_origin,Time,weight
0,ID_001,TRO-05432,2014-06-05,C. albicans,1A,1,1,9.0,Tinfection,100.0
1,ID_001,TRO-05433,2014-06-05,C. albicans,1A,1,1,9.0,Tinfection,100.0
2,ID_001,TRO-05434,2014-06-05,C. albicans,1A,1,1,9.0,Tinfection,100.0
3,ID_001,TRO-05435,2014-06-05,C. albicans,1A,1,1,6.0,Tinfection,100.0
4,ID_001,TRO-05456,2014-06-05,C. albicans,1A,1,1,7.0,Tinfection,100.0
...,...,...,...,...,...,...,...,...,...,...
32993,ID_096,TRO-028337,2023-03-03,S. pneumoniae,3,3,1,5.0,T13,NaN
32994,ID_096,TRO-028338,2023-03-03,S. pneumoniae,3,3,1,4.0,T13,NaN
32995,ID_096,TRO-028339,2023-03-03,S. pneumoniae,3,3,1,6.0,T13,NaN
32996,ID_096,TRO-028342,2023-03-03,S. pneumoniae,3,3,0,8.0,T13,NaN


### cleaning datas for computation

In [36]:
df_clean = df.copy()
status_mapping = {0: 'Alive', 1: 'Dead'}
df_clean["survival_original"] = df_clean['survival_original'].replace(status_mapping)

infection_mapping = {'Listeria':"L. monocytogenes"}
df_clean['Infection'] =df_clean['Infection'].replace(infection_mapping)

df_clean.rename(columns={'survival_original':'Mice'},inplace=True)

time_rename = list(np.arange(0,15,1))
time_unique = df_clean['Time'].unique()
time_mapping = dict(zip(time_unique,time_rename))
df_clean['Time'] = df_clean['Time'].replace(time_mapping)
df_clean


,ID_Experiment,Mouse_ID,Date,Infection,Group,exp,Mice,t_origin,Time,weight
0,ID_001,TRO-05432,2014-06-05,C. albicans,1A,1,Dead,9.0,0,100.0
1,ID_001,TRO-05433,2014-06-05,C. albicans,1A,1,Dead,9.0,0,100.0
2,ID_001,TRO-05434,2014-06-05,C. albicans,1A,1,Dead,9.0,0,100.0
3,ID_001,TRO-05435,2014-06-05,C. albicans,1A,1,Dead,6.0,0,100.0
4,ID_001,TRO-05456,2014-06-05,C. albicans,1A,1,Dead,7.0,0,100.0
...,...,...,...,...,...,...,...,...,...,...
32993,ID_096,TRO-028337,2023-03-03,S. pneumoniae,3,3,Dead,5.0,13,NaN
32994,ID_096,TRO-028338,2023-03-03,S. pneumoniae,3,3,Dead,4.0,13,NaN
32995,ID_096,TRO-028339,2023-03-03,S. pneumoniae,3,3,Dead,6.0,13,NaN
32996,ID_096,TRO-028342,2023-03-03,S. pneumoniae,3,3,Alive,8.0,13,NaN


### time to event

In [37]:
kmf = KaplanMeierFitter()
# A4 page dimensions in inches

df_KP = df_clean.copy()
df_KP['Mice'] = df_KP['Mice'].replace({'Dead':1,'Alive':0})

kmf.fit(df_KP["t_origin"], df_KP["Mice"], label='Total')
median_ = kmf.median_survival_time_
median_confidence_interval_ = median_survival_times(kmf.confidence_interval_)
print(median_)
print(median_confidence_interval_)

for name, grouped_df in df_KP.groupby('Infection'):
    print(name)
    kmf.fit(grouped_df["t_origin"], grouped_df["Mice"], label=name)
    median_ = kmf.median_survival_time_
    median_confidence_interval_ = median_survival_times(kmf.confidence_interval_)
    print(median_)
    print(median_confidence_interval_)


11.0
     Total_lower_0.95  Total_upper_0.95
0.5              11.0              11.0
C. albicans
inf
     C. albicans_lower_0.95  C. albicans_upper_0.95
0.5                     inf                     inf
H1N1
inf
     H1N1_lower_0.95  H1N1_upper_0.95
0.5              inf              inf
L. monocytogenes
6.0
     L. monocytogenes_lower_0.95  L. monocytogenes_upper_0.95
0.5                          6.0                          6.0
S. pneumoniae
9.0
     S. pneumoniae_lower_0.95  S. pneumoniae_upper_0.95
0.5                       8.0                       9.0


In [38]:
kmf = KaplanMeierFitter()

df_KP = df_clean.copy()
df_KP['Mice'] = df_KP['Mice'].replace({'Dead':1,'Alive':0})
df_dead = df_KP[df_KP['Mice']==1]

kmf.fit(df_dead["t_origin"], df_dead["Mice"], label='Total')
median_ = kmf.median_survival_time_
median_confidence_interval_ = median_survival_times(kmf.confidence_interval_)
print(median_)
print(median_confidence_interval_)

for name, grouped_df in df_dead.groupby('Infection'):
    kmf.fit(grouped_df["t_origin"], grouped_df["Mice"], label=name)
    median_ = kmf.median_survival_time_
    median_confidence_interval_ = median_survival_times(kmf.confidence_interval_)
    print(median_)
    print(median_confidence_interval_)


5.0
     Total_lower_0.95  Total_upper_0.95
0.5               5.0               5.0
8.0
     C. albicans_lower_0.95  C. albicans_upper_0.95
0.5                     8.0                     8.0
7.0
     H1N1_lower_0.95  H1N1_upper_0.95
0.5              7.0              7.0
4.0
     L. monocytogenes_lower_0.95  L. monocytogenes_upper_0.95
0.5                          4.0                          4.0
5.0
     S. pneumoniae_lower_0.95  S. pneumoniae_upper_0.95
0.5                       5.0                       5.0


In [39]:
df = pd.read_excel("./data/df_for_analysis.xlsx",index_col=0)
df.groupby('Infection')['time_original'].mean()


Infection
C. albicans      11.472222
H1N1              9.764881
Listeria          5.665076
S. pneumoniae     7.554785
Name: time_original, dtype: float64

In [40]:
df['time_original'].mean()

7.448451421298261